<a href="https://colab.research.google.com/github/Ezhil2318/Pandas_Practice/blob/main/pandasPractice_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandasql
import pandas as pd
from pandasql import sqldf
from datetime import timedelta,datetime

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=1f01032e5762ad51699c90a5c382ff237f1eb5a9f7e2a91b3c7082fdbb4e4e94
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [ ]:
customers_df = pd.DataFrame({
    'customer_id': [101, 102, 103, 104, 105],
    'name': ['Amit', 'Beena', 'Carlos', 'Deepa', 'Elena'],
    'region_code': ['N1', 'S2', 'N1', 'W3', 'S2'],
    'age': [34, 45, 29, 51, 38],
    'risk_score': [0.12, 0.30, 0.09, 0.40, 0.25]
})

In [ ]:
transactions_df = pd.DataFrame({
    'transaction_id': [2001, 2002, 2003, 2004, 2005, 2006],
    'customer_id': [101, 102, 101, 104, 105, 102],
    'amount': [9500, 12000, 7000, 5000, None, 13000],
    'timestamp': pd.to_datetime([
        '2025-01-15', '2025-01-20', '2025-02-10', '2025-02-28', '2025-03-01', '2025-03-15'
    ]),
    'channel': ['Online', 'ATM', 'Branch', 'Branch', 'Online', 'ATM']
})

In [ ]:
loans_df = pd.DataFrame({
    'loan_id': [301, 302, 303, 304],
    'customer_id': [101, 102, 103, 105],
    'loan_type': ['Home', 'Auto', 'Credit', 'Home'],
    'status': ['Active', 'Closed', 'Active', 'Active'],
    'issue_date': pd.to_datetime(['2023-12-01', '2022-05-15', '2024-07-01', '2023-09-10'])
})

In [ ]:
payments_df = pd.DataFrame({
    'payment_id': [5001, 5002, 5003, 5004, 5005, 5006],
    'loan_id': [301, 301, 303, 303, 303, 304],
    'payment_date': pd.to_datetime([
        '2025-01-05', '2025-02-05', '2025-01-15', '2025-02-15', '2025-03-15', '2025-02-20'
    ]),
    'payment_amount': [1000, 1000, None, 800, None, 1200]
})

In [ ]:
accounts_df = pd.DataFrame({
    'account_id': [601, 602, 603, 604],
    'region': ['N1', 'S2', 'W3', 'N1'],
    'balance': [15000, 8000, 12500, 7000],
    'portfolio_mgr': ['Raj', 'Meera', 'Raj', 'Anil']
})

In [ ]:
transactions_df['avg_transaction'] = transactions_df.groupby('customer_id')['amount'].transform('mean')
transactions_df['frequency'] = transactions_df.groupby('customer_id')['transaction_id'].transform('count')
condition1 = transactions_df['avg_transaction'].quantile(0.95)
condition2 = transactions_df['frequency'].quantile(0.95)
transactions_df[(transactions_df.avg_transaction>=condition1)&(transactions_df.frequency>=condition2)]

,transaction_id,customer_id,amount,timestamp,channel,avg_transaction,frequency
1,2002,102,12000.0,2025-01-20,ATM,12500.0,2
5,2006,102,13000.0,2025-03-15,ATM,12500.0,2


In [ ]:
a = loans_df.merge(payments_df,on='loan_id',how='inner')
sixM = pd.to_datetime('today').normalize() - pd.DateOffset(months=6)
b = a[a.payment_date>sixM]
b['null_counts']= b.groupby('customer_id')['payment_amount'].transform(lambda x:x.isna().sum())
c = customers_df.merge(b,on='customer_id',how='inner')
c[c.null_counts>2].loc[:,['customer_id','name']].drop_duplicates()

/tmp/ipython-input-8-937016000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['null_counts']= b.groupby('customer_id')['payment_amount'].transform(lambda x:x.isna().sum())


,customer_id,name


In [ ]:
merged = customers_df.merge(transactions_df,on='customer_id',how='outer',indicator=True)

In [ ]:
# Step 1: Create month column
transactions_df['month'] = transactions_df['timestamp'].dt.to_period('M')

# Step 2: Aggregate spend per month
monthly_spend = transactions_df.groupby('month')['amount'].sum().to_timestamp()

# Step 3: Add rolling average and delta
monthly_spend_df = monthly_spend.to_frame(name='total_spend')
monthly_spend_df['rolling_avg'] = monthly_spend_df['total_spend'].rolling(window=2).mean()
monthly_spend_df['delta'] = monthly_spend_df['total_spend'].diff()

monthly_spend_df

,total_spend,rolling_avg,delta
month,,,
2025-01-01,21500.0,NaN,NaN
2025-02-01,12000.0,16750.0,-9500.0
2025-03-01,13000.0,12500.0,1000.0


In [ ]:
# Check for duplicate (loan_id, payment_date) pairs
duplicate_flags = payments_df.duplicated(subset=['loan_id', 'payment_date'], keep=False)

# Merge with loans to get context
flagged = payments_df[duplicate_flags].merge(loans_df, on='loan_id', how='left')
flagged[['loan_id', 'payment_date', 'customer_id', 'payment_amount']]

,loan_id,payment_date,customer_id,payment_amount


In [ ]:
# Extract date
transactions_df['date'] = transactions_df['timestamp'].dt.date

# Daily amount per customer
daily_totals = transactions_df.groupby(['customer_id', 'date'])['amount'].sum().reset_index()

# Compute z-score per customer
daily_totals['z_score'] = daily_totals.groupby('customer_id')['amount'].transform(
    lambda x: (x - x.mean()) / x.std()
)

# Outliers: z-score above threshold
outliers = daily_totals[daily_totals['z_score'] > 3]
outliers.sort_values(by='z_score', ascending=False)

,customer_id,date,amount,z_score


In [ ]:
# Create pivot for category-wise spend per customer
pivot = transactions_df.pivot_table(
    index='customer_id',
    columns='channel',
    values='amount',
    aggfunc='sum',
    fill_value=0
)

pivot.head()

channel,ATM,Branch,Online
customer_id,,,
101,0.0,7000.0,9500.0
102,25000.0,0.0,0.0
104,0.0,5000.0,0.0
105,0.0,0.0,0.0
